Import all the required libraries

In [ ]:
# Import the necessary libraries
import ee
import pandas as pd
import geemap

Initialize and authenticate Google Earth Engine API

In [ ]:
# Initialize and Authenticate Google Earth Engine API
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Define the region of interest using the drawing tool

In [ ]:
# Create a basemap for interactive mapping
Map = geemap.Map(center=[31.511122, 74.35684], zoom=9)
Map

Load the shapefile and convert it to an Earth Engine feature collection

In [ ]:
# Defining the Region of Interest using the drawing tool
area = ee.FeatureCollection(Map.draw_last_feature)
roi = ee.Geometry.Point([73.996, 31.725])
geometry = geemap.shp_to_ee('/path/to/Your_Shapefile.shp')

Filtering Image Collection and Clipping to ROI

In [ ]:
# Filtering the Image Collection according to our Region of Interest
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
filtered = ee.Image(L8.filterDate('2015-01-01', '2015-12-31')
            .filterBounds(roi)
            .sort('CLOUD_COVER'))
            
filtered_image = filtered.clip(geometry)
RGB_VIS = {'min': 0, 'max': 1, 'bands': ['B4', 'B3', 'B2']}
Map.addLayer(filtered_image, RGB_VIS, 'True Color')

Calculating Normalized Difference Vegetation Index (NDVI)

In [ ]:
####### Calculating NDVI
NDVI_VIS = {'min': 0, 'max': 1, 'pallete': ['e74c3c', 'f1948a', 'f7dc6f', 'abebc6', '117a65']}

# Calculating NDVI
nir = filtered_image.select('B5')
red = filtered_image.select('B4')
NDVI = (nir.subtract(red).divide(nir.add(red)))

#### Adding NDVI layer on the interactive map
Map.addLayer(NDVI, NDVI_VIS, 'NDVI')

Function to Calculate LST

In [ ]:
def process_image(image):
    """
    Processes a Landsat 8 image and adds a band for land surface temperature (LST).
    
    Parameters:
    image (ee.Image): A Landsat 8 image.
    
    Returns:
    ee.Image: The input image with an added LST band.
    """
    # Calculate Top of Atmosphere Radiance
    toa = image.expression(
        '0.00033420 * therm_band + 0.1000 - 0.29',
        {'therm_band': image.select('B10')}
    )

    # Calculate Brightness Temperature
    bt = image.expression(
        '1321.0789 / log(774.8853 / radiance + 1) - 273.15',
        {'radiance': toa}
    )

    # Calculate NDVI
    ndvi = image.expression(
        '(band5 - band4) / (band5 + band4)',
        {'band5': image.select('B5'), 'band4': image.select('B4')}
    ).rename('NDVI')

    # Calculate the Minimum NDVI for the Region
    min_ndvi = (
        ndvi.reduceRegion(
            reducer=ee.Reducer.min(),
            geometry=geometry,
            scale=30,
            maxPixels=1e13
        )
        .get('NDVI')
    )

    # Calculate the Maximum NDVI for the Region
    max_ndvi = (
        ndvi.reduceRegion(
            reducer=ee.Reducer.max(),
            geometry=geometry,
            scale=30,
            maxPixels=1e13
        )
        .get('NDVI')
    )

    # Calculate Proportion of Vegetation
    pv = (
        (ndvi - min_ndvi) / (max_ndvi - min_ndvi)
    ).pow(ee.Number(2)).rename('PV')

    # Calculate Land surface Emissivity
    a = ee.Number(0.004)
    b = ee.Number(0.986)
    em = (pv * a + b).rename('LSE')

    # Calculate land surface temperature
    lst = image.expression(
    '(Tb/(1 + (0.00115* (Tb / 14380))*log(EM))) -273.15', {
    'Tb': image.select('B10'),
    'EM': em.select('LSE')
    }).rename('LST')

  # Return the image with the temperature band added
    return image.addBands(lst)

lst = process_image(imag)
final_lst = lst.select('LST').getInfo()
LST_VIS = {'min': 10, 'max': 60, 'pallete': ['e74c3c', 'f1948a', 'f7dc6f', 'abebc6', '117a65']}
Map.addLayer(lst, LST_VIS, 'LST')
